# GPT-3.5-Turbo Model
Creating a question answering chatbot using GPT-3.5. Adapted from: https://github.com/openai/openai-cookbook/blob/main/examples/Question_answering_using_embeddings.ipynb

num_tokens() IS OFTEN USED IN KNOWLEDGE.PY (and other files??) WITHOUT SPECIFYING THE EMBEDDING MODEL.

In [ ]:
import sys
sys.path.append("modules")
from modules.config import *
from modules.knowledge import *
from modules.chatbot import *
from modules.embedding_functions import *
from modules.query import *
sys.path.remove("modules")

In [ ]:
filename = 'assets/knowledge/Fundamentals_of_Digital_Image_Processing_Textbook.pdf'

In [ ]:
CompVisionKnowledge = Knowledge('CompVisionPDF', 'GPT')
CompVisionKnowledge.append_pdf(filename, 'CompVisionPDF') # Need to debug this!

In [40]:
doc = fitz.open(filename)
output = []
def get_blank_knowledge_df() -> pd.DataFrame:
    return pd.DataFrame(columns=['Source', 'Heading', 'Subheading', 'Content'])
document_name = 'Comp Vision Stuff'
page_limit = 31
content = get_blank_knowledge_df()
for page in doc:
    page_limit = doc.page_count if not page_limit else page_limit
    if page.number <= page_limit:
        block_content = page.get_text("blocks")
        for block in block_content:
                if block[6] == 0: # I.e. only extract text
                    plain_text = unidecode(block[4])
                    new_row = {'Source': document_name, 'Heading': page.number, 'Content': plain_text}
                    content = pd.concat([content, pd.DataFrame.from_records([new_row])])
    else:
        pass
import re
# cleaned_text = re.sub(r'http\S+', '', all_text)
# cleaned_text = re.sub(r'www.+', '', cleaned_text)
content['Content'] = content['Content'].apply(lambda x: re.sub(r'http\S+', '', x))
content['Content'] = content['Content'].apply(lambda x: re.sub(r'www.+', '', x))
content = content.loc[content['Content']!='\n']
content

,Source,Heading,Subheading,Content
0,Comp Vision Stuff,0,NaN,GLOBAL \nEDITION\n
0,Comp Vision Stuff,0,NaN,Digital Image Processing\n
0,Comp Vision Stuff,0,NaN,FOURTH EDITION\n
0,Comp Vision Stuff,0,NaN,Rafael C. Gonzalez * Richard E. Woods\n
0,Comp Vision Stuff,1,NaN,Support Package for Digital \nImage Processing\n
...,...,...,...,...
0,Comp Vision Stuff,31,NaN,7\nShort-wave infrared\n2.09-2.35\nMineral map...
0,Comp Vision Stuff,31,NaN,TABLE 1.1\nThematic bands \nof NASA's \nLANDSA...
0,Comp Vision Stuff,31,NaN,1\n2\n3\n
0,Comp Vision Stuff,31,NaN,4\n5\n6\n7\n


In [29]:
content

,Source,Heading,Subheading,Content
0,Comp Vision Stuff,{0},NaN,\nGLOBAL \nEDITION\n
0,Comp Vision Stuff,{0},NaN,\nDigital Image Processing\n
0,Comp Vision Stuff,{0},NaN,\n FOURTH EDITION\n
0,Comp Vision Stuff,{0},NaN,\n Rafael C. Gonzalez * Richard E. Woods\n
0,Comp Vision Stuff,{0},NaN,\nwww.EBooksWorld.ir\n
...,...,...,...,...
0,Comp Vision Stuff,{1021},NaN,\n18 Chapter 1 Introduction\n
0,Comp Vision Stuff,{1021},NaN,\n1.1 WHAT IS DIGITAL IMAGE PROCESSING? \n
0,Comp Vision Stuff,{1021},NaN,\nAn image may be defined as a two-dimensional...
0,Comp Vision Stuff,{1021},NaN,\n1.1\n


In [23]:
text2 = 'My name is Joel and my website is www.joelpointon.ir'
cleaned_text = re.sub(r'http\S+', '', all_text)
cleaned_text = re.sub(r'www.+', '', cleaned_text)
cleaned_text

'GLOBAL \nEDITION\nDigital Image Processing\n FOURTH EDITION\n Rafael C. Gonzalez * Richard E. Woods\n\nYour new textbook provides access to support packages that may include reviews in areas \nlike probability and vectors, tutorials on topics relevant to the material in the book, an image \ndatabase, and more. Refer to the Preface in the textbook for a detailed list of resources.\nFollow the instructions below to register for the Companion Website for Rafael C. Gonzalez and \nRichard E. Woods\' Digital Image Processing, Fourth Edition, Global Edition.\n1. Go to \n2. Find the title of your textbook.\n3.  Click Support Materials and follow the on-screen instructions to create a login name and \npassword.\nUse the login name and password you created during registration to start using the \ndigital resources that accompany your textbook.\nIMPORTANT:\nThis serial code can only be used once. This subscription is not transferrable.\n\nProcessing\nigital Image\n4\nD\nF O U R T H\nE D I T I O 

In [15]:
from langchain.text_splitter import TokenTextSplitter
class Article:
    def count_tokens(self, text):
        return num_tokens(text)

    def split_document(self, content):
        CHUNK_SIZE = 20
        from langchain.text_splitter import RecursiveCharacterTextSplitter
        # text_splitter = TokenTextSplitter(chunk_size=10, chunk_overlap=0, encoding_name=GPT_EMBEDDING_MODEL) # , separators=['\n\n', '\n', '.', ' ', '']
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=CHUNK_SIZE,
                                                       length_function=num_tokens,
                                                       separators=['\n\n', '\n', '.', ' ', ''], # ['', ' ', '.', '\n', '\n\n'],
                                                       chunk_overlap=5)
        return text_splitter.split_text(content)

In [16]:
inst = Article()
chunks = inst.split_document(all_text)

AttributeError: 'Knowledge' object has no attribute 'append_pdf'

In [ ]:
output_file_folder = 'assets/'

In [16]:
from unidecode import unidecode # parsing symbols
import fitz
doc = fitz.open(filename)
page_num = 0
page_numbers = []
content = []
output = []
for page in doc:
    page_num += 1
    blocks = page.get_text("blocks")
    for block in blocks:
        if block[6]==0:
            block_content = unidecode(block[4])
            stripped_block_content = block_content.replace('\n', '')
            if not stripped_block_content.isdigit() and not is_float(stripped_block_content):
                content.append(block_content)
                page_numbers.append(page_num)
            else:
                pass
content_df = pd.DataFrame(
    {'Page Numbers': page_numbers,
     'Content': content,
    })
content_df['no_newlines'] = content_df['Content'].str.replace(r'\n', '', regex=True)
content_df

,Page Numbers,Content,no_newlines
0,1,GLOBAL \nEDITION\n,GLOBAL EDITION
1,1,Digital Image Processing\n,Digital Image Processing
2,1,FOURTH EDITION\n,FOURTH EDITION
3,1,Rafael C. Gonzalez * Richard E. Woods\n,Rafael C. Gonzalez * Richard E. Woods
4,1,www.EBooksWorld.ir\n,www.EBooksWorld.ir
...,...,...,...
19376,1022,This is a special edition of an established \n...,This is a special edition of an established ti...
19377,1022,Pearson Global Edition\n,Pearson Global Edition
19378,1022,GLOBAL \nEDITION\n,GLOBAL EDITION
19379,1022,"For these Global Editions, the editorial team ...","For these Global Editions, the editorial team ..."


In [18]:
block_dict = {}
page_num = 1
for page in doc: # Iterate all pages in the document
      file_dict = page.get_text('dict') # Get the page dictionary
      block = file_dict['blocks'] # Get the block information
      block_dict[page_num] = block # Store in block dictionary
      page_num += 1 # Increase the page value by 1

In [5]:
target_url = 'https://homepages.inf.ed.ac.uk/rbf/HIPR2/median.htm'
import urllib.request  # the lib that handles the url stuff

for line in urllib.request.urlopen(target_url):
    print(line.decode('utf-8')) #utf-8 or iso8859-1 or whatever the page encoding scheme is

URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1129)>

In [13]:
CompVisionKnowledge = Knowledge(WIKI_PAGE, 'GPT')
for page in WIKI_PAGES:
    CompVisionKnowledge.append_wikipedia_page(WIKI_PAGE)
# save document chunks and embeddings
CompVisionKnowledge.export_to_csv(GPT_KNOWLEDGE_FILENAME)
CompVisionKnowledge.df

,Source,Heading,Subheading,Content,Section,Tokens,Embedding
0,Wikipedia (Computer vision),,,\nComputer vision tasks include methods for ac...,Wikipedia (Computer vision),286,"[-0.01913553662598133, 0.002932898933067918, 0..."
1,Wikipedia (Computer vision),Definition,,\nComputer vision is an interdisciplinary fiel...,Wikipedia (Computer vision)->Definition,158,"[-0.021093836054205894, 0.0049119978211820126,..."
2,Wikipedia (Computer vision),History,,"\nIn the late 1960s, computer vision began at ...",Wikipedia (Computer vision)->History,507,"[-0.011549791321158409, -0.004044382367283106,..."
4,Wikipedia (Computer vision),Related fields,Solid-state physics,\nSolid-state physics is another field that is...,Wikipedia (Computer vision)->Related fields->S...,120,"[0.0018743288237601519, 0.011324070394039154, ..."
5,Wikipedia (Computer vision),Related fields,Neurobiology,\nNeurobiology has greatly influenced the deve...,Wikipedia (Computer vision)->Related fields->N...,293,"[-0.009132628329098225, 0.0011366719845682383,..."
6,Wikipedia (Computer vision),Related fields,Signal processing,\nYet another field related to computer vision...,Wikipedia (Computer vision)->Related fields->S...,103,"[-0.027298789471387863, 0.007510432507842779, ..."
7,Wikipedia (Computer vision),Related fields,Robotic navigation,\nRobot navigation sometimes deals with autono...,Wikipedia (Computer vision)->Related fields->R...,64,"[0.0034529592376202345, -0.014102335087954998,..."
8,Wikipedia (Computer vision),Related fields,Other fields,\nBesides the above-mentioned views on compute...,Wikipedia (Computer vision)->Related fields->O...,119,"[0.002435609931126237, -0.003915637265890837, ..."
9,Wikipedia (Computer vision),Related fields,Distinctions,\nThe fields most closely related to computer ...,Wikipedia (Computer vision)->Related fields->D...,639,"[-0.017207426950335503, 0.005905073136091232, ..."
10,Wikipedia (Computer vision),Applications,,\nApplications range from tasks such as indust...,Wikipedia (Computer vision)->Applications,272,"[-0.022458024322986603, 0.005672922823578119, ..."


In [20]:
CompVisionKnowledgeBERT = Knowledge(WIKI_PAGE, 'BERT')
CompVisionKnowledgeBERT.append_wikipedia_page(WIKI_PAGE)
# save document chunks and embeddings
CompVisionKnowledgeBERT.export_to_csv(BERT_KNOWLEDGE_FILENAME)
CompVisionKnowledgeBERT.df

,Source,Heading,Subheading,Content,Section,Tokens,Embedding
0,Wikipedia (Computer vision),,,\nComputer vision tasks include methods for ac...,Wikipedia (Computer vision),290,"[-0.5566069483757019, 0.6151323318481445, 0.70..."
1,Wikipedia (Computer vision),Definition,,\nComputer vision is an interdisciplinary fiel...,Wikipedia (Computer vision)->Definition,162,"[-0.18940897285938263, 0.5564344525337219, 0.5..."
2,Wikipedia (Computer vision),History,,"\nIn the late 1960s, computer vision began at ...",Wikipedia (Computer vision)->History,243,"[-0.5624328255653381, 0.35494446754455566, 0.6..."
3,Wikipedia (Computer vision),History,,"\nBy the 1990s, some of the previous research ...",Wikipedia (Computer vision)->History,264,"[-0.8420819044113159, 0.011862404644489288, 0...."
5,Wikipedia (Computer vision),Related fields,Solid-state physics,\nSolid-state physics is another field that is...,Wikipedia (Computer vision)->Related fields->S...,123,"[-0.1766018569469452, 0.5509802103042603, 0.11..."
6,Wikipedia (Computer vision),Related fields,Neurobiology,\nNeurobiology has greatly influenced the deve...,Wikipedia (Computer vision)->Related fields->N...,296,"[0.07454751431941986, 0.42800015211105347, 0.1..."
7,Wikipedia (Computer vision),Related fields,Signal processing,\nYet another field related to computer vision...,Wikipedia (Computer vision)->Related fields->S...,106,"[-0.1562240570783615, 0.18830031156539917, 0.4..."
8,Wikipedia (Computer vision),Related fields,Robotic navigation,\nRobot navigation sometimes deals with autono...,Wikipedia (Computer vision)->Related fields->R...,65,"[0.09209920465946198, 0.5207788944244385, 1.26..."
9,Wikipedia (Computer vision),Related fields,Other fields,\nBesides the above-mentioned views on compute...,Wikipedia (Computer vision)->Related fields->O...,122,"[-0.34635502099990845, 0.12120083719491959, 0...."
10,Wikipedia (Computer vision),Related fields,Distinctions,\nThe fields most closely related to computer ...,Wikipedia (Computer vision)->Related fields->D...,244,"[-0.011814514175057411, 0.9892112612724304, 0...."


# Search
Now we'll define a search function that:

Takes a user query and a dataframe with text & embedding columns
Embeds the user query with the OpenAI API
Uses distance between query embedding and text embeddings to rank the texts
Returns two lists:
The top N texts, ranked by relevance
Their corresponding relevance scores

In [19]:
# CompVisionBERT = ChatBot("Computer Vision", 'assets/' + BERT_KNOWLEDGE_FILENAME)
# print(Query.ask_bert('When did universities begin teaching Computer Vision?', CompVisionBERT))

# CompVisionGPT = ChatBot("Computer Vision", 'assets/' + GPT_KNOWLEDGE_FILENAME)
# print(Query.ask('Who is Boris Johnson', CompVisionGPT, show_source=True))

CompVisionGPT = ChatBot("Computer Vision", 'assets/' + GPT_KNOWLEDGE_FILENAME)
print(Query.ask_bart('When did Universities begin teaching Computer Vision?', CompVisionGPT, show_source=True)) # What if the GPT knowledge sections are longer than 1024 tokens?? Need to account for this!

# Todo:
# I need to make it more efficient on the number of tokens.
# Adapt it for more sources (e.g. PDF)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [39]:
CompVisionGPT = ChatBot("Computer Vision", 'assets/' + GPT_KNOWLEDGE_FILENAME)
print(Query.ask('Who is Boris Johnson', CompVisionGPT, show_source=True))

C:\Users\point\AppData\Local\Temp\ipykernel_23968\3384959035.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.knowledge_used['Index'] = np.arange(len(self.knowledge_used))+1
C:\Users\point\AppData\Local\Temp\ipykernel_23968\3384959035.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.knowledge_used['Cumulative_tokens'] = self.knowledge_used['Tokens'].cumsum()
C:\Users\point\AppData\Local\Temp\ipykernel_23968\3384959035.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of

I could not find an answer in the text I've been provided, sorry! Please try again.

Total tokens used: 721


In [13]:
CompVisionBERT = ChatBot("Computer Vision", 'assets/' + BERT_KNOWLEDGE_FILENAME)
print(Query.ask_bert('When did universities begin teaching Computer Vision?', CompVisionBERT))

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

ValueError: operands could not be broadcast together with shapes (768,) (1536,) 

In [20]:
CompVisionGPT = ChatBot("Computer Vision", 'assets/' + GPT_KNOWLEDGE_FILENAME)
print(Query.ask_bart('When did Universities begin teaching Computer Vision?', CompVisionGPT, show_source=True))

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [19]:
import torch

def describe(x):
    print("Type: {}".format(x.type()))
    print("Shape/size: {}".format(x.shape))
    print("Values: \n{}".format(x))

describe(torch.Tensor(2, 3))

Type: torch.FloatTensor
Shape/size: torch.Size([2, 3])
Values: 
tensor([[6.7410e+22, 2.6729e+23, 5.3689e-05],
        [1.3542e-05, 5.2905e-08, 6.7942e-07]])


# Tasks
Clean the parsed text so we have pure english words

In [31]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

from nltk.tokenize import word_tokenize
text = "It's Here’s to the crazy ones, the misfits, the rebels, the troublemakers, the round pegs in the square holes. The ones who see things differently — they’re not fond of rules. You can quote them, disagree with them, glorify or vilify them, but the only thing you can’t do is ignore them because they change things. They push the human race forward, and while some may see them as the crazy ones, we see genius, because the ones who are crazy enough to think that they can change the world, are the ones who do."
tokens = word_tokenize(text)

# Remove stopwords
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))
filtered_tokens = [token for token in tokens if token.lower() not in stop_words]
print(filtered_tokens)

["'s", '’', 'crazy', 'ones', ',', 'misfits', ',', 'rebels', ',', 'troublemakers', ',', 'round', 'pegs', 'square', 'holes', '.', 'ones', 'see', 'things', 'differently', '—', '’', 'fond', 'rules', '.', 'quote', ',', 'disagree', ',', 'glorify', 'vilify', ',', 'thing', '’', 'ignore', 'change', 'things', '.', 'push', 'human', 'race', 'forward', ',', 'may', 'see', 'crazy', 'ones', ',', 'see', 'genius', ',', 'ones', 'crazy', 'enough', 'think', 'change', 'world', ',', 'ones', '.']


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\point\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\point\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [32]:
tokens

['It',
 "'s",
 'Here',
 '’',
 's',
 'to',
 'the',
 'crazy',
 'ones',
 ',',
 'the',
 'misfits',
 ',',
 'the',
 'rebels',
 ',',
 'the',
 'troublemakers',
 ',',
 'the',
 'round',
 'pegs',
 'in',
 'the',
 'square',
 'holes',
 '.',
 'The',
 'ones',
 'who',
 'see',
 'things',
 'differently',
 '—',
 'they',
 '’',
 're',
 'not',
 'fond',
 'of',
 'rules',
 '.',
 'You',
 'can',
 'quote',
 'them',
 ',',
 'disagree',
 'with',
 'them',
 ',',
 'glorify',
 'or',
 'vilify',
 'them',
 ',',
 'but',
 'the',
 'only',
 'thing',
 'you',
 'can',
 '’',
 't',
 'do',
 'is',
 'ignore',
 'them',
 'because',
 'they',
 'change',
 'things',
 '.',
 'They',
 'push',
 'the',
 'human',
 'race',
 'forward',
 ',',
 'and',
 'while',
 'some',
 'may',
 'see',
 'them',
 'as',
 'the',
 'crazy',
 'ones',
 ',',
 'we',
 'see',
 'genius',
 ',',
 'because',
 'the',
 'ones',
 'who',
 'are',
 'crazy',
 'enough',
 'to',
 'think',
 'that',
 'they',
 'can',
 'change',
 'the',
 'world',
 ',',
 'are',
 'the',
 'ones',
 'who',
 'do',
 '.']

In [150]:
sentences = [
"Artificial intelligence (AI) is revolutionising industries and transforming the way we live and work.",
"Machine learning, a subset of AI, enables computers to learn and make predictions or decisions without being explicitly programmed.",
"AI algorithms can analyse vast amounts of data to uncover patterns, detect anomalies, and extract valuable insights.",
"Natural language processing (NLP) allows machines to understand and process human language, facilitating communication between humans and computers.",
"Computer vision enables machines to interpret and understand visual information, enabling applications such as image recognition and object detection.",
"AI-powered virtual assistants like Siri, Alexa, and Google Assistant are becoming increasingly common, providing voice-based interactions and personalised experiences.",
"AI is driving advancements in healthcare, including disease diagnosis, drug discovery, and personalised medicine, leading to improved patient outcomes.",
"In the field of autonomous vehicles, AI plays a crucial role in enabling self-driving cars to perceive their surroundings and make informed decisions.",
"AI is transforming the customer service industry by utilising chatbots and automated systems to provide faster and more efficient support to customers.",
"Ethical considerations, such as transparency, fairness, and privacy, are essential in the development and deployment of AI systems to ensure responsible and accountable use.",
"Cars have revolutionised transportation, providing a convenient and efficient means of travel for people around the world.",
"Automobile manufacturing involves a complex process of designing, engineering, and assembling various components to create a functional vehicle.",
"Safety features such as seat belts, airbags, and anti-lock braking systems have greatly improved the overall safety of cars.",
"Electric vehicles (EVs) are gaining popularity as eco-friendly alternatives to traditional petrol-powered cars, reducing carbon emissions and dependence on fossil fuels.",
"Advanced driver-assistance systems (ADAS) enhance car safety by incorporating technologies like adaptive cruise control and lane-keeping assist.",
"Sports cars are known for their high-performance capabilities, offering speed, agility, and an exhilarating driving experience.",
"Classic cars hold a special place in automotive history, with their timeless designs and nostalgic appeal capturing the hearts of car enthusiasts.",
"Car customisation allows owners to personalise their vehicles, from unique paint jobs and body modifications to performance upgrades.",
"Car-sharing services and ride-hailing apps have transformed the way people access transportation, providing convenient alternatives to car ownership.",
"The future of cars is expected to bring autonomous vehicles, where cars can navigate and operate without human intervention, promising increased safety and efficiency."
]

In [152]:
import gensim
from gensim.models import Word2Vec
from tqdm import tqdm

def preprocess_text(text: str, remove_stopwords: bool) -> str:
    """Function that cleans the input text by going to:
    - remove links
    - remove special characters
    - remove numbers
    - remove stopwords
    - convert to lowercase
    - remove excessive white spaces
    Arguments:
        text (str): text to clean
        remove_stopwords (bool): whether to remove stopwords
    Returns:
        str: cleaned text
    """
    # remove links
    text = re.sub(r"http\S+", "", text)
    # remove numbers and special characters
    text = re.sub("[^A-Za-z]+", " ", text)
    # remove stopwords
    if remove_stopwords:
        # 1. create tokens
        tokens = nltk.word_tokenize(text)
        # 2. check if it's a stopword
        tokens = [w.lower().strip() for w in tokens if not w.lower() in stopwords.words("english")]
        # return a list of cleaned tokens
        return tokens

tokenised_sentences = [preprocess_text(sentence, remove_stopwords=True) for sentence in sentences]

In [153]:
tokenised_sentences

[['artificial',
  'intelligence',
  'ai',
  'revolutionising',
  'industries',
  'transforming',
  'way',
  'live',
  'work'],
 ['machine',
  'learning',
  'subset',
  'ai',
  'enables',
  'computers',
  'learn',
  'make',
  'predictions',
  'decisions',
  'without',
  'explicitly',
  'programmed'],
 ['ai',
  'algorithms',
  'analyse',
  'vast',
  'amounts',
  'data',
  'uncover',
  'patterns',
  'detect',
  'anomalies',
  'extract',
  'valuable',
  'insights'],
 ['natural',
  'language',
  'processing',
  'nlp',
  'allows',
  'machines',
  'understand',
  'process',
  'human',
  'language',
  'facilitating',
  'communication',
  'humans',
  'computers'],
 ['computer',
  'vision',
  'enables',
  'machines',
  'interpret',
  'understand',
  'visual',
  'information',
  'enabling',
  'applications',
  'image',
  'recognition',
  'object',
  'detection'],
 ['ai',
  'powered',
  'virtual',
  'assistants',
  'like',
  'siri',
  'alexa',
  'google',
  'assistant',
  'becoming',
  'increasing

In [154]:
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')
lemmatizer = WordNetLemmatizer()

from nltk.corpus import wordnet

# function to convert nltk tag to wordnet tag
def get_wordnet_pos(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN # Default to noun if the POS tag is unknown/ambiguous

def lemmatize_sentence(tokenised_sentence):
    tagged_tokens = nltk.pos_tag(tokenised_sentence)
    lemmatized_words = []
    for word, tag in tagged_tokens:
        pos = get_wordnet_pos(tag)
        lemmatized_words.append(lemmatizer.lemmatize(word, pos=pos))
    return lemmatized_words

# refined_sentences = [sentence.strip()]
lemmatised_sentences = [lemmatize_sentence(s) for s in tokenised_sentences]
lemmatised_sentences

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\point\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\point\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\point\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


[['artificial',
  'intelligence',
  'ai',
  'revolutionise',
  'industry',
  'transform',
  'way',
  'live',
  'work'],
 ['machine',
  'learn',
  'subset',
  'ai',
  'enable',
  'computer',
  'learn',
  'make',
  'prediction',
  'decision',
  'without',
  'explicitly',
  'program'],
 ['ai',
  'algorithm',
  'analyse',
  'vast',
  'amount',
  'data',
  'uncover',
  'pattern',
  'detect',
  'anomaly',
  'extract',
  'valuable',
  'insight'],
 ['natural',
  'language',
  'processing',
  'nlp',
  'allow',
  'machine',
  'understand',
  'process',
  'human',
  'language',
  'facilitate',
  'communication',
  'human',
  'computer'],
 ['computer',
  'vision',
  'enable',
  'machine',
  'interpret',
  'understand',
  'visual',
  'information',
  'enable',
  'application',
  'image',
  'recognition',
  'object',
  'detection'],
 ['ai',
  'power',
  'virtual',
  'assistant',
  'like',
  'siri',
  'alexa',
  'google',
  'assistant',
  'become',
  'increasingly',
  'common',
  'provide',
  'voice'

In [155]:
from gensim.models import Word2Vec

VECTOR_SIZE = 100
MIN_COUNT = 1
WINDOW = 3
SG = 1

model = Word2Vec(
    sentences=lemmatised_sentences,
    vector_size=VECTOR_SIZE,
    min_count=MIN_COUNT,
    sg=SG
)

model = Word2Vec(sentences=lemmatised_sentences, min_count=1)

In [157]:
model.wv.similarity('ai', 'algorithm')

# model.wv.most_similar(positive=['ai'], topn=5)

-0.052282296

In [107]:
from sklearn.manifold import TSNE
def reduce_dimensions(model):
    num_components = 2  # number of dimensions to keep after compression

    # extract vocabulary from model and vectors in order to associate them in the graph
    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)

    # apply TSNE
    tsne = TSNE(n_components=num_components, random_state=0)
    vectors = tsne.fit_transform(vectors)

    x_vals = [v[0] for v in vectors]
    y_vals = [v[1] for v in vectors]
    return x_vals, y_vals, labels


def plot_embeddings(x_vals, y_vals, labels):
    import plotly.graph_objs as go
    fig = go.Figure()
    trace = go.Scatter(x=x_vals, y=y_vals, mode='markers', text=labels)
    fig.add_trace(trace)
    fig.update_layout(title="Word2Vec - Visualising embeddings with TSNE")
    fig.show()
    return fig


x_vals, y_vals, labels = reduce_dimensions(model)

plot = plot_embeddings(x_vals, y_vals, labels)

C:\Users\point\anaconda3\lib\site-packages\sklearn\manifold\_t_sne.py:780: FutureWarning:

The default initialization in TSNE will change from 'random' to 'pca' in 1.2.

C:\Users\point\anaconda3\lib\site-packages\sklearn\manifold\_t_sne.py:790: FutureWarning:

The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.



In [158]:
from transformers import AutoTokenizer, GPT2Model
import torch

tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = GPT2Model.from_pretrained("gpt2")

inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")
outputs = model(**inputs)

last_hidden_states = outputs.last_hidden_state